In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torch.utils.data as Data,DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from prefetch_generator import BackgroundGenerator

import torch_geometric.nn as gnn
import torch_geometric.data as gdata
import torch_geometric.nn.models as gmodel
from torch_geometric.data import Data,DataLoader
from torch_geometric.nn import GraphConv, TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

import pandas as pd
import numpy as np
from tqdm import tqdm

import os
import matplotlib
import matplotlib.pyplot as plt 

import hiddenlayer as hl

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

#准备文件夹保存数据
local_path=os.getcwd()
if not os.path.exists(local_path+'\\blokagedata'):
    os.mkdir(local_path+'\\blokagedata')
print(local_path+'\\blokagedata')
#判断是否可以使用GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class DataLoaderX(DataLoader):
    def __iter__(self):
        return BackgroundGenerator(super().__iter__())
def run_model(batch,model):
    model.eval()
    torch.no_grad()
    prediction=model(batch)
    return prediction.detach()
def score_model(batch,model):
    x=run_model(batch,model)
    x=torch.argmax(x,1)
    score=(x==batch.y).sum().item()/len(x)
    return round(score,4)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = gnn.GATConv(1, 1,heads=3)
#         self.conv2 = gnn.GCNConv(3, 3)
#         self.conv3 = gnn.SAGEConv(1, 1)
#         self.pool1 = TopKPooling(100, ratio=0.4)
#         self.pool2 = TopKPooling(100, ratio=0.4)
        self.lin1 = nn.Linear(96*3, 200)
        self.lin2 = nn.Linear(200, 117)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
#         x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
#         x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
#         x = F.relu(self.conv2(x, edge_index))
#         x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
#         x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
#         x= x1 + x2
#         x = F.relu(self.conv2(x, edge_index))
#         x = F.relu(self.conv3(x, edge_index))
#         x = torch.mean(x,1)
        x = x.reshape([int(len(x)/96),96*3])
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        return x

In [ ]:
traindata=pd.read_csv(local_path+'\\blokagedata\\sample_117.csv',header=0).iloc[:,range(633,729)]
trainlabel=np.array(pd.read_csv(local_path+'\\blokagedata\\y_117.csv',header=0)).astype(int)
edge_index=pd.read_excel(local_path+'\\blokagedata\\edge_index.xls',header=0)
x_train,x_test,y_train,y_test=train_test_split(traindata,trainlabel,test_size=0.2,random_state=1,stratify=trainlabel)
scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train).T
x_test=scaler.transform(x_test).T
e=np.array(edge_index).astype(int).T
train_list=[]
for i in range(len(y_train)):
    X=torch.tensor(x_train[:,i]).unsqueeze(1).float()
    Y=torch.tensor(y_train[i,0]).long()
    E=torch.tensor(e).long()
    data = Data(x=X, edge_index=E,y=Y)
    train_list.append(data)
train_loader= DataLoader(train_list, batch_size=round(len(y_train)*1), shuffle=True)#round(len(y_train)*0.25)
test_list=[]
for i in range(len(y_test)):
    X=torch.tensor(x_test[:,i]).unsqueeze(1).float()
    Y=torch.tensor(y_test[i,0]).long()
    E=torch.tensor(e).long()
    data = Data(x=X, edge_index=E,y=Y)
    test_list.append(data)
test_loader= DataLoader(test_list, batch_size=len(y_test), shuffle=True)

In [ ]:
model = Net().to(device)
criterion=nn.CrossEntropyLoss().to(device)
optimizer = Adam(model.parameters(), lr=0.0001,weight_decay=0.001)
# scheduler=LambdaLR(optimizer,verbose=False,lr_lambda=lambda i:1/(i/750+1))
acc=0
i=0
while acc<100.0 :
    i+=1
    for one_train_batch in train_loader:
        one_train_batch=one_train_batch.to(device)
        model.train()
        prediction = model(one_train_batch)
        loss =criterion(prediction, one_train_batch.y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#     scheduler.step()
    for test_batch in test_loader:
        test_batch=test_batch.to(device)
        acc=round(score_model(test_batch,model),4)*100
    print ("epoch: {}\t, loss: {:.6f}, test_acc: {}%".format(i, loss.item(), acc))